# Алгоритм шифрования на эллиптических кривых

## Введение

В данной работе реализован алгоритм шифрования, основанный на использовании эллиптических кривых. Эллиптическая криптография является одним из самых мощных инструментов в области современной криптографии, обеспечивающим высокий уровень безопасности при относительно малых размерах ключей по сравнению с классическими методами.

Эллиптические кривые в криптографии представляют собой множество точек, удовлетворяющих уравнению вида $y^2 = x^3 + ax + b$ в конечном поле. Основные операции, выполняемые с точками на кривой, включают сложение точек и умножение точек на число.

## Описание алгоритма

Алгоритм шифрования на эллиптических кривых включает в себя следующие шаги:

1. **Выбор параметров кривой и генерирующей точки** $G$. В нашем случае использовалась кривая с параметрами $a = -1$, $b = 1$ и модулем $p = 751$, генерирующая точка $G = (0, 1)$.

2. **Генерация пары ключей**. Секретный ключ $nb$ является случайным большим числом. Открытый ключ $Pb$ вычисляется как $nb \times G$.

3. **Шифрование сообщения**. Каждый символ сообщения сопоставляется с точкой на кривой. Для шифрования символа используется пара точек $(C1, C2)$, где $C1 = kG$, $C2 = Pm + kPb$, $Pm$ — точка, соответствующая символу, $k$ — случайное число для каждого символа.

4. **Дешифрование сообщения**. Получатель может восстановить исходную точку $Pm$ как $C2 - nbC1$, используя свой секретный ключ $nb$.

## Реализация

Реализация алгоритма включает определение функций для арифметических операций на эллиптических кривых (сложение точек, удвоение точек, умножение точки на число) и функций для шифрования/дешифрования сообщений. Также реализована функция для генерации пары ключей.

## Выводы

Реализация алгоритма шифрования на эллиптических кривых показывает, как с помощью математических свойств эллиптических кривых можно создать эффективную систему шифрования. Этот метод обеспечивает высокий уровень безопасности и может быть использован в современных криптографических системах для защиты информации.

In [180]:
import random

# Параметры эллиптической кривой и модуля
a = -1
b = 1
p = 751
G = (0, 1)


# Вспомогательные функции
def modinv(a, p):
    return pow(a, p - 2, p)


def point_add(P, Q):
    if P == (0, 0): return Q
    if Q == (0, 0): return P
    if P == Q: return point_double(P)
    xp, yp = P
    xq, yq = Q
    if xp == xq and yp == p - yq: return (0, 0)
    lam = (yq - yp) * modinv(xq - xp, p) % p
    x = (lam ** 2 - xp - xq) % p
    y = (lam * (xp - x) - yp) % p
    return (x, y)


def point_double(P):
    if P == (0, 0): return P
    xp, yp = P
    lam = (3 * xp ** 2 + a) * modinv(2 * yp, p) % p
    x = (lam ** 2 - 2 * xp) % p
    y = (lam * (xp - x) - yp) % p
    return (x, y)


def point_multiply(P, n):
    R = (0, 0)
    N = P
    while n:
        if n & 1: R = point_add(R, N)
        N = point_double(N)
        n >>= 1
    return R


# Генерация ключей
def generate_keys():
    nb = random.randint(1, p - 1)
    Pb = point_multiply(G, nb)
    return nb, Pb


# Шифрование
def encrypt(Pm, k, Pb):
    C1 = point_multiply(G, k)
    C2 = point_add(Pm, point_multiply(Pb, k))
    return C1, C2


# Функция для инвертирования точки не предоставлена, добавьте ее согласно вашим требованиям
def point_negate(P):
    x, y = P
    return (x, p - y) if y != 0 else P


# Дешифрование
def decrypt(C1, C2, nb):
    return point_add(C2, point_negate(point_multiply(C1, nb)))


def decrypt_to_text(decrypted_points, alphabet):
    # Инвертирование алфавита для поиска символов по точкам
    points_to_char = {point: char for char, point in alphabet.items()}
    # Сопоставление каждой точки с символом
    return ''.join(points_to_char.get(point, '?') for point in decrypted_points)


alphabet = {'л': (240, 399), 'а': (93, 484), 'т': (247, 266),
            'ы': (247, 485), 'ш': (236, 399), 'с': (243, 664), 'к': (105, 382), 'и': (102, 484),
            'й': (236, 712)}  # и так далее для всего алфавита

message = 'латышский'

# Пример использования
nb, Pb = generate_keys()
k_values = [random.randint(1, p - 1) for _ in range(len(message))]
ciphertext = [encrypt(alphabet[char], k, Pb) for char, k in zip(message, k_values)]
decrypted_points = [decrypt(C1, C2, nb) for C1, C2 in ciphertext]
decrypted_message = decrypt_to_text(decrypted_points, alphabet)
print("Исходное сообщение:", message)
print("Исходный алфавит:", alphabet)
print("Секретный ключ (nb):", nb)
print("Открытый ключ (Pb):", Pb)
print("Шифротекст:", ciphertext)
print("Полученные точки:", decrypted_points)
print("Полученные текст:", decrypted_message)

Исходное сообщение: латышский
Исходный алфавит: {'л': (240, 399), 'а': (93, 484), 'т': (247, 266), 'ы': (247, 485), 'ш': (236, 399), 'с': (243, 664), 'к': (105, 382), 'и': (102, 484), 'й': (236, 712)}
Секретный ключ (nb): 44
Открытый ключ (Pb): (175, 192)
Шифротекст: [((56, 332), (433, 684)), ((568, 396), (304, 638)), ((725, 195), (48, 49)), ((181, 618), (25, 147)), ((750, 1), (533, 303)), ((16, 335), (426, 89)), ((702, 225), (458, 261)), ((16, 416), (115, 509)), ((82, 270), (229, 600))]
Полученные точки: [(240, 399), (93, 484), (247, 266), (247, 485), (236, 399), (243, 664), (105, 382), (102, 484), (236, 712)]
Полученные текст: латышский


In [175]:
G = (-1, 1)  # Обновленная генерирующая точка

# Секретный ключ
nb = 25

# Шифротекст (пример)
ciphertext = [
    ((72, 254), (397, 184)),
    ((188, 93), (526, 412)),
    ((188, 93), (328, 290)),
    ((135, 82), (433, 47)),
    ((179, 275), (711, 341)),
    ((568, 355), (546, 670)),
    ((16, 416), (734, 170)),
    ((568, 355), (371, 14)),
    ((596, 433), (604, 610)),
    ((16, 416), (734, 170))
]

# Дешифрование сообщения
decrypted_points = [decrypt(C1, C2, nb) for C1, C2 in ciphertext]

alphabet = {' ': (33, 355), 'б': (228, 480), 'а': (228, 271),
            'д': (234, 164), 'м': (238, 175), 'и': (236, 39), 'н': (238, 576), 'т': (247, 266),
            'о': (240, 309)}  # и так далее для всего алфавита

print("Дешифрованные точки:", decrypted_points)
print("Расшифрованное сообщение:", decrypt_to_text(decrypted_points, alphabet))


Дешифрованные точки: [(33, 355), (228, 480), (228, 271), (234, 164), (238, 175), (236, 39), (238, 576), (247, 266), (240, 309), (238, 576)]
Расшифрованное сообщение:  бадминтон
